<a href="https://colab.research.google.com/github/Nabetse1109/uao_riesgo_credito/blob/main/notebooks/01_data_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto: Gestión del riesgo y cobranza mediante modelos predictivos en créditos educativos**

Autores: Paula Andrea Tovar Ríos, Edgar Esteban Grajales Castaño

Entorno: Google Colab - Python

Importación de librerías:

In [63]:
# Importación de librerías

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Configuración gráfica
plt.rcParams["figure.figsize"] = (10, 5)
sns.set(style="whitegrid", font_scale=1.0)

# Para que se vean todas las columnas cuando inspeccionemos dataframes
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


Ruta:

In [31]:
# Conexión a Google Drive y carga de datasets

from google.colab import drive
import pandas as pd

# Montar Google Drive
drive.mount('/content/drive')

# Ruta base del proyecto en tu Drive
BASE_DIR = "/content/drive/MyDrive/uao_riesgo_credito"

# Subcarpetas
DATA_RAW_DIR = f"{BASE_DIR}/data/raw"
DATA_PROCESSED_DIR = f"{BASE_DIR}/data/processed"

# Paths completos de los datasets
CREDITOS_PATH = f"{DATA_RAW_DIR}/1_Creditos_Estudiantes.xlsx"
CARTERA_PATH = f"{DATA_RAW_DIR}/2_Cartera_depurada.xlsx"
PAGARES_PATH = f"{DATA_RAW_DIR}/3_Info_Pagares.xlsx"

CREDITOS_PATH, CARTERA_PATH, PAGARES_PATH



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/uao_riesgo_credito/data/raw/1_Creditos_Estudiantes.xlsx',
 '/content/drive/MyDrive/uao_riesgo_credito/data/raw/2_Cartera_depurada.xlsx',
 '/content/drive/MyDrive/uao_riesgo_credito/data/raw/3_Info_Pagares.xlsx')

Carga de datasets:

In [32]:
# Cargar los datasets en dataframes

print("Cargando datasets...")

creditos_df = pd.read_excel(CREDITOS_PATH)
cartera_df  = pd.read_excel(CARTERA_PATH)
pagares_df  = pd.read_excel(PAGARES_PATH)

print("Datasets cargados correctamente:")
print(f" - Créditos: {creditos_df.shape}")
print(f" - Cartera depurada: {cartera_df.shape}")
print(f" - Info Pagares: {pagares_df.shape}")



Cargando datasets...
Datasets cargados correctamente:
 - Créditos: (405829, 30)
 - Cartera depurada: (27330, 66)
 - Info Pagares: (522286, 10)


Estilo de las tablas:

In [33]:
# Función para dar estilo a las tablas

def estilo_tabla(df, titulo=None):
    """
    Aplica estilo a un DataFrame para visualización en Colab / Jupyter.
    - Encabezados oscuros
    - Bordes sutiles
    - Nulos resaltados
    - Título tipo caption
    """
    # Copia para evitar modificar el original
    df_to_style = df.copy()

    # Arrancamos el styler
    styled = df_to_style.style

    # Caption (título)
    if titulo:
        styled = styled.set_caption(titulo)

    # Estilos de la tabla
    styled = styled.set_table_styles([
        {
            "selector": "caption",
            "props": [
                ("color", "#2c3e50"),
                ("font-size", "16px"),
                ("font-weight", "bold"),
                ("text-align", "left"),
                ("margin-bottom", "10px")
            ]
        },
        {
            "selector": "th",
            "props": [
                ("background-color", "#2c3e50"),
                ("color", "white"),
                ("font-weight", "bold"),
                ("border", "1px solid #ddd"),
                ("padding", "5px")
            ]
        },
        {
            "selector": "td",
            "props": [
                ("border", "1px solid #ddd"),
                ("padding", "5px")
            ]
        }
    ])

    # Alineación
    styled = styled.set_properties(**{"text-align": "left"})

    # Resaltar nulos
    styled = styled.highlight_null(color="#f8d7da")

    # Formatear columnas numéricas (separador de miles)
    numeric_cols = df_to_style.select_dtypes(include="number").columns
    if len(numeric_cols) > 0:
        fmt_dict = {col: "{:,.0f}".format for col in numeric_cols}
        styled = styled.format(fmt_dict, na_rep="-")

    return styled




Vista rápida primeras filas:

In [34]:
# Primer vistazo a los datos (head)

# Créditos
estilo_tabla(creditos_df.head(), titulo="Primeras filas del dataset de Créditos")


,Cliente,Tipo_identificacion,ID_Estudiante,Credito,Contar,Linea_credito,Descripcion_linea_credito,Nota_debito,Concepto_nota_debito,Nombre_concepto,Causa_nota_debito,Nombre_causa_nota,Valor_nota_debito,Valor_pagado_nota_debito,Fecha_pago_nota_debito,Saldo_nota_debito,% Recaudo,Fecha_vencimiento_ndb,Periodo_facturacion,Mes_facturacion,Periodo_academico,Documento,Estado,Estado_describe,Centro_costo,Nombre_centro_costo,Fecha_solicitud,Fecha_aprobacion,Cuotas,Periodo
0,"2,030,623",CC,E00001,1,0,1,PREGRADO NORMAL DE 1 A 3 SEMESTRE,"10,044",99,CREDITO EDUCATIVO UAO,204,CUOTA CAPITAL CREDITO UAO.- PREGRADO,"600,000","600,000",2003-01-21 00:00:00,0,100,2002-12-21 00:00:00,-,-,2003-01,FAP,N,CANCELADO,"2,009",PROGRAMA FUNDAMENTACION,2003-01-21 00:00:00,2002-12-21 00:00:00,4,2003-1S
1,"2,030,623",CC,E00001,1,0,1,PREGRADO NORMAL DE 1 A 3 SEMESTRE,"10,046",99,CREDITO EDUCATIVO UAO,204,CUOTA CAPITAL CREDITO UAO.- PREGRADO,"175,000","175,000",2003-02-21 00:00:00,0,100,2003-01-21 00:00:00,-,-,2003-01,FAP,N,CANCELADO,"2,009",PROGRAMA FUNDAMENTACION,2003-01-21 00:00:00,2002-12-21 00:00:00,4,2003-1S
2,"2,030,623",CC,E00001,1,0,1,PREGRADO NORMAL DE 1 A 3 SEMESTRE,"10,047",99,CREDITO EDUCATIVO UAO,204,CUOTA CAPITAL CREDITO UAO.- PREGRADO,"175,000","175,000",2003-04-21 00:00:00,0,100,2003-02-21 00:00:00,-,-,2003-01,FAP,N,CANCELADO,"2,009",PROGRAMA FUNDAMENTACION,2003-01-21 00:00:00,2002-12-21 00:00:00,4,2003-1S
3,"2,030,623",CC,E00001,1,0,1,PREGRADO NORMAL DE 1 A 3 SEMESTRE,"10,048",99,CREDITO EDUCATIVO UAO,204,CUOTA CAPITAL CREDITO UAO.- PREGRADO,"175,000","175,000",2003-04-21 00:00:00,0,100,2003-03-21 00:00:00,-,-,2003-01,FAP,N,CANCELADO,"2,009",PROGRAMA FUNDAMENTACION,2003-01-21 00:00:00,2002-12-21 00:00:00,4,2003-1S
4,"2,030,623",CC,E00001,1,1,1,PREGRADO NORMAL DE 1 A 3 SEMESTRE,"10,049",99,CREDITO EDUCATIVO UAO,204,CUOTA CAPITAL CREDITO UAO.- PREGRADO,"175,000","175,000",2003-06-20 00:00:00,0,100,2003-04-21 00:00:00,-,-,2003-01,FAP,N,CANCELADO,"2,009",PROGRAMA FUNDAMENTACION,2003-01-21 00:00:00,2002-12-21 00:00:00,4,2003-1S


In [35]:
# Cartera depurada
estilo_tabla(cartera_df.head(), titulo="Primeras filas del dataset de Cartera Depurada")

,CLIENTE,TIPO ID,ID_Estudiante,# REG,EDAD CARTERA,NOMBRE EDAD,DÍAS,RANGO,SALDO,ORGANIZACIÓN,TIPO CLIENTE,NOMBRE TIPO CLIENTE,TIPO CARTERA,TIPO CARTERA 2,ACTIVA / INACTIVA,TIPO DOCUMENTO,NOMBRE TIPO DOCUMENTO,DOCUMENTO,NOMBRE DOCUMENTO,NÚMERO CRÉDITO,CONTROL SALDO,GRUPO,CONCEPTO NOTA,NOMBRE CONCEPTO,CAUSA NOTA,NOMBRE CAUSA NOTA,CLASIFICADOR,REFERENCIA,NOMBRE REFERENCIA,CONCEPTO,FECHA,FECHA VENCE,PERIODO,NOMBRE PERIODO,ORGANIZACIÓN LIQUIDACIÓN,DOCUMENTO LIQUIDACIÓN,LIQUIDACIÓN ORDEN,ORGANIZACIÓN CENTRO,CENTRO COSTO,NOMBRE CENTRO COSTO,FONDO,NOMBRE FONDO,FUENTE FUNCIÓN,NOMBRE FUENTE FUNCIÓN,DESCRIPCIÓN,VALOR TOTAL,VALOR DESCUENTO,VALOR BRUTO,GENERA MORA,TASA INTERÉS MORA,FECHA LIQUIDACIÓN INTERÉS,INICIO DIFERIDO,MESES DIFERIDO,FECHA DOCUMENTO,FECHA VENCIMIENTO,VALOR DOCUMENTO,VALOR AFECTADO,FECHA CANCELACIÓN,ORGANIZACIÓN ORIGEN,DOCUMENTO ORIGEN,NÚMERO ORIGEN,ESTADO,CRÉDITO,CUOTA,CONCEPTO CRÉDITO,NOMBRE CONCEPTO CRÉDITO
0,"931,360",TI,E26331,8,8,SUPERIOR DE 1080 DÍAS,"1,280",9. MAS DE 1080 DIAS,"12,853",1,1,ESTUDIANTES,VENCIDA,CORTO PLAZO,INA,F,FACTURACION,ODP,OTROS DERECHOS PECUNIARIOS,"12,981",-,-,-,-,-,-,"72,325","72,325",CERTIFICADOS ACADEMICOS PREGRADO,72325-CERTIFICADOS ACADEMICOS PREGRADO,2022-01-27 00:00:00,2022-02-28 00:00:00,2022-1,PECUNIARIOS SOLICITADOS EN PRIMER SEMESTRE DEL AÑO 2022,1,ODP,"12,981",1,"2,068",PROGRAMA DE ECONOMIA,11,FONDO GENERAL DE OPERACIÓN UAO,10,INGRESOS ACADEMICOS,SOLICITUD CERTIFICADO ACADÉMICO..MAM,"15,600",0,"15,600",N,0,-,2022-01-26 00:00:00,1,2022-01-27 00:00:00,2022-02-28 00:00:00,"15,600","2,747",-,-,-,-,V,-,-,-,-
1,"22,500,842",CE,E25699,2,10,DE 31 A 60 DÍAS POR VENCER,-33,1. AL DIA,"1,233,600",1,1,ESTUDIANTES,POR VENCER,CORTO PLAZO,ACT,D,DEBITO,NDB,NOTA DEBITO,"2,958,037",T,17,515,CUOTA DE CAPITAL CRÉDITO EDUCATIVO UAO POSGRADO,338,CRÉDITO UAO LINEA(30) - CP,-,-,-,515/338 CRÉDITO UAO LINEA(30) - CP,2025-07-04 00:00:00,2025-10-03 00:00:00,20252S,SEGUNDO SEMESTRE ACADEMICO DEL AÑO 2025,1,FAPO,"24,566",1,"2,088",MAESTRÍA EN INTELIGENCIA ARTIFICIAL Y CIENCIA DE DATOS - COMBINADA 2025-1,11,FONDO GENERAL DE OPERACIÓN UAO,10,INGRESOS ACADEMICOS,NOTA GENERADA PARA EL CREDITO: 197273. JANGARITA - PORTAL WEB - ASOCIA Y CONFIRMA. EN LA CUOTA: 3 POR EL CONCEPTO: PAGO CUOTA DE CAPITAL,"1,233,600",0,"1,233,600",S,0,-,2025-10-03 00:00:00,1,2025-07-04 00:00:00,2025-10-03 00:00:00,"1,233,600",0,-,-,-,-,V,"197,273",3,C,CAPITAL
2,"22,500,842",CE,E25699,3,11,DE 61 A 90 DÍAS POR VENCER,-66,1. AL DIA,"1,233,600",1,1,ESTUDIANTES,POR VENCER,CORTO PLAZO,ACT,D,DEBITO,NDB,NOTA DEBITO,"2,958,038",T,17,515,CUOTA DE CAPITAL CRÉDITO EDUCATIVO UAO POSGRADO,338,CRÉDITO UAO LINEA(30) - CP,-,-,-,515/338 CRÉDITO UAO LINEA(30) - CP,2025-07-04 00:00:00,2025-11-05 00:00:00,20252S,SEGUNDO SEMESTRE ACADEMICO DEL AÑO 2025,1,FAPO,"24,566",1,"2,088",MAESTRÍA EN INTELIGENCIA ARTIFICIAL Y CIENCIA DE DATOS - COMBINADA 2025-1,11,FONDO GENERAL DE OPERACIÓN UAO,10,INGRESOS ACADEMICOS,NOTA GENERADA PARA EL CREDITO: 197273. JANGARITA - PORTAL WEB - ASOCIA Y CONFIRMA. EN LA CUOTA: 4 POR EL CONCEPTO: PAGO CUOTA DE CAPITAL,"1,233,600",0,"1,233,600",S,0,-,2025-11-05 00:00:00,1,2025-07-04 00:00:00,2025-11-05 00:00:00,"1,233,600",0,-,-,-,-,V,"197,273",4,C,CAPITAL
3,"22,500,842",CE,E25699,4,12,DE 91 A 180 DÍAS POR VENCER,-96,1. AL DIA,"1,233,600",1,1,ESTUDIANTES,POR VENCER,CORTO PLAZO,ACT,D,DEBITO,NDB,NOTA DEBITO,"2,958,039",T,17,515,CUOTA DE CAPITAL CRÉDITO EDUCATIVO UAO POSGRADO,338,CRÉDITO UAO LINEA(30) - CP,-,-,-,515/338 CRÉDITO UAO LINEA(30) - CP,2025-07-04 00:00:00,2025-12-05 00:00:00,20252S,SEGUNDO SEMESTRE ACADEMICO DEL AÑO 2025,1,FAPO,"24,566",1,"2,088",MAESTRÍA EN INTELIGENCIA ARTIFICIAL Y CIENCIA DE DATOS - COMBINADA 2025-1,11,FONDO GENERAL DE OPERACIÓN UAO,10,INGRESOS ACADEMICOS,NOTA GENERADA PARA EL CREDITO: 197273. JANGARITA - PORTAL WEB - ASOCIA Y CONFIRMA. EN LA CUOTA: 5 POR EL CONCEPTO: PAGO CUOTA DE CAPITAL,"1,233,600",0,"1,233,600",S,0,-,2025-12-05 00:00:00,1,2025-07-04 00:00:00,2025-12-05 00:00:00,"1,233,600",0,-,-,-,-,V,"197,

In [36]:
# Info pagares
estilo_tabla(pagares_df.head(), titulo="Primeras filas del dataset de Info pagares")


,ID CRÉDITO,ID_Estudiante,Sexo,Estrato,Departamento,Pais Nacimiento,Primiparo,Estado_est,Graduado,Cargo_codeudor
0,1,E00001,FEM,0,VALLE DEL CAUCA,COLOMBIA,-,-,-,-
1,2,E00002,MAS,0,VALLE DEL CAUCA,COLOMBIA,-,-,-,Coo. de Seguridad
2,3,E00003,MAS,0,VALLE DEL CAUCA,COLOMBIA,-,-,-,-
3,4,E00004,MAS,0,VALLE DEL CAUCA,COLOMBIA,-,-,-,-
4,5,E00005,MAS,6,VALLE DEL CAUCA,COLOMBIA,-,-,-,-


Información general de variables:

In [37]:
# Tabla resumen de info para Créditos

info_creditos = pd.DataFrame({
    "columna": creditos_df.columns,
    "tipo_dato": creditos_df.dtypes.astype(str),
    "n_nulos": creditos_df.isna().sum(),
    "%_nulos": (creditos_df.isna().mean() * 100).round(2)
})

# Ordenamos por % de nulos descendente para ver primero las variables más problemáticas
info_creditos = info_creditos.sort_values("%_nulos", ascending=False).reset_index(drop=True)

estilo_tabla(info_creditos, titulo="Información general del dataset de Créditos")


,columna,tipo_dato,n_nulos,%_nulos
0,Periodo_facturacion,object,"47,231",12
1,Mes_facturacion,object,"47,231",12
2,Documento,object,"41,123",10
3,Fecha_pago_nota_debito,datetime64[ns],"29,478",7
4,Cliente,int64,0,0
5,Tipo_identificacion,object,0,0
6,Descripcion_linea_credito,object,0,0
7,Nota_debito,int64,0,0
8,Concepto_nota_debito,int64,0,0
9,Nombre_concepto,object,0,0


In [38]:
# Tabla resumen de info para Cartera Depurada

info_cartera = pd.DataFrame({
    "columna": cartera_df.columns,
    "tipo_dato": cartera_df.dtypes.astype(str),
    "n_nulos": cartera_df.isna().sum(),
    "%_nulos": (cartera_df.isna().mean() * 100).round(2)
})

info_cartera = info_cartera.sort_values("%_nulos", ascending=False).reset_index(drop=True)

estilo_tabla(info_cartera, titulo="Información general del dataset de Cartera Depurada")


,columna,tipo_dato,n_nulos,%_nulos
0,FECHA CANCELACIÓN,datetime64[ns],"26,886",98
1,DOCUMENTO ORIGEN,object,"26,263",96
2,ORGANIZACIÓN ORIGEN,float64,"26,263",96
3,NÚMERO ORIGEN,float64,"26,263",96
4,NOMBRE REFERENCIA,object,"25,561",94
5,CLASIFICADOR,float64,"25,561",94
6,REFERENCIA,float64,"25,561",94
7,FECHA LIQUIDACIÓN INTERÉS,object,"25,085",92
8,GRUPO,float64,"17,153",63
9,DOCUMENTO LIQUIDACIÓN,object,"13,809",51


In [39]:
# Tabla resumen de calidad de datos — Info Pagares

info_pagares = pd.DataFrame({
    "columna": pagares_df.columns,
    "tipo_dato": pagares_df.dtypes.astype(str),
    "n_nulos": pagares_df.isna().sum(),
    "%_nulos": (pagares_df.isna().mean() * 100).round(2)
})

info_pagares = info_pagares.sort_values("%_nulos", ascending=False).reset_index(drop=True)

estilo_tabla(info_pagares, titulo="Información general del dataset de Info Pagares")


,columna,tipo_dato,n_nulos,%_nulos
0,Cargo_codeudor,object,"203,648",39
1,Graduado,object,"37,965",7
2,Primiparo,object,"37,965",7
3,Estado_est,object,"37,966",7
4,ID_Estudiante,object,"33,558",6
5,Departamento,object,"23,530",5
6,Estrato,float64,"23,530",5
7,Pais Nacimiento,object,"23,530",5
8,ID CRÉDITO,int64,0,0
9,Sexo,object,0,0


Al revisar las tablas de valores nulos en los dos conjuntos de datos (Créditos y Cartera depurada), podemos darnos cuenta de que algunas columnas tienen muchos datos faltantes (nulos), mientras que otras están casi completas.

En el caso de Créditos, hay columnas como `Periodo_facturacion` y `Mes_facturacion` que tienen aproximadamente un 12% de valores nulos, lo que significa que en uno de cada diez registros falta esa información. Otras columnas, como `Documento` y `Fecha_pago_nota_debito`, también tienen un porcentaje menor de datos faltantes, pero no tan crítico.

En Cartera depurada, la situación es aún más notable porque encontramos variables como `FECHA CANCELACIÓN`, `DOCUMENTO ORIGEN` y `NOMBRE REFERENCIA` con más del 90% de valores nulos. Esto quiere decir que, para la mayoría de los registros, esa información no está disponible.

Estos resultados nos indican dos cosas principales:

Hay ciertas variables que tal vez no sean tan relevantes para el análisis, porque apenas tienen información.

En otras, aunque el porcentaje de nulos es menor, deberíamos considerar cómo manejar esos valores faltantes, ya sea completándolos de alguna manera (si es posible) o descartando esas columnas o registros, dependiendo de la importancia que tengan para los modelos o el análisis que queremos hacer.

En resumen, identificar estos valores nulos desde el comienzo nos ayuda a entender la calidad de nuestros datos y decidir, de manera informada, los siguientes pasos para limpiarlos y prepararlos correctamente antes de analizarlos o utilizarlos en modelos predictivos.

Resumen tamaño de los datasets:

In [40]:
# Resumen rápido de tamaño de los datasets

resumen_shapes = pd.DataFrame({
    "dataset": ["Créditos", "Cartera depurada", "Info Pagares"],
    "n_filas":  [
        creditos_df.shape[0],
        cartera_df.shape[0],
        pagares_df.shape[0]
    ],
    "n_columnas": [
        creditos_df.shape[1],
        cartera_df.shape[1],
        pagares_df.shape[1]
    ]
})

estilo_tabla(resumen_shapes, titulo="Dimensión de los datasets")



,dataset,n_filas,n_columnas
0,Créditos,"405,829",30
1,Cartera depurada,"27,330",66
2,Info Pagares,"522,286",10


Corrección de tipos de datos:

In [41]:
# Corrección de tipos de datos en creditos_df

# Limpieza ligera de nombres de columnas (quita espacios)
creditos_df.columns = creditos_df.columns.str.strip()

# Conversión explícita de columnas de fecha
cols_fecha = [
    "Fecha_pago_nota_debito",
    "Fecha_vencimiento_ndb",
    "Fecha_solicitud",
    "Fecha_aprobacion",
]

for col in cols_fecha:
    if col in creditos_df.columns:
        creditos_df[col] = pd.to_datetime(
            creditos_df[col],
            errors="coerce",
            dayfirst=True
        )

# Conversión de columnas numéricas (enteros y floats)

cols_int = [
    "Cliente",
    "Credito",
    "Contar",
    "Nota_debito",
    "Concepto_nota_debito",
    "Causa_nota_debito",
    "Centro_costo",
    "Cuotas",
]

cols_float = [
    "Valor_nota_debito",
    "Valor_pagado_nota_debito",
    "Saldo_nota_debito",
    "% Recaudo",
]

for col in cols_int:
    if col in creditos_df.columns:
        creditos_df[col] = pd.to_numeric(creditos_df[col], errors="coerce").astype("Int64")

for col in cols_float:
    if col in creditos_df.columns:
        creditos_df[col] = pd.to_numeric(creditos_df[col], errors="coerce")

# Conversión de columnas categóricas (incluye IDs anónimos)

cols_categoricas = [
    "Tipo_identificacion",
    "ID_Estudiante",
    "Linea_credito",
    "Descripcion_linea_credito",
    "Nombre_concepto",
    "Nombre_causa_nota",
    "Estado",
    "Estado_describe",
    "Nombre_centro_costo",
]

for col in cols_categoricas:
    if col in creditos_df.columns:
        creditos_df[col] = creditos_df[col].astype("category")

# Tratamiento de columnas de periodo (las dejamos como categóricas)

cols_periodo = [
    "Periodo_facturacion",
    "Mes_facturacion",
    "Periodo_academico",
    "Periodo",
]

for col in cols_periodo:
    if col in creditos_df.columns:
        creditos_df[col] = creditos_df[col].astype("category")

# Documento: mantener como texto (object) o pasarlo a categoría si quieres
if "Documento" in creditos_df.columns:
    creditos_df["Documento"] = creditos_df["Documento"].astype("string")  # texto limpio
    # Si más adelante quieres tratarlo como categórico:
    # creditos_df["Documento"] = creditos_df["Documento"].astype("category")

print("Corrección de tipos completada.")


Corrección de tipos completada.


In [42]:
tipos_creditos_post = pd.DataFrame({
    "columna": creditos_df.columns,
    "tipo_dato": creditos_df.dtypes.astype(str)
}).reset_index(drop=True)

estilo_tabla(tipos_creditos_post, titulo="Tipos de datos después de la corrección – Créditos")


,columna,tipo_dato
0,Cliente,Int64
1,Tipo_identificacion,category
2,ID_Estudiante,category
3,Credito,Int64
4,Contar,Int64
5,Linea_credito,category
6,Descripcion_linea_credito,category
7,Nota_debito,Int64
8,Concepto_nota_debito,Int64
9,Nombre_concepto,category


In [43]:
# Corrección de tipos de datos en cartera_df

# Limpieza ligera de nombres de columnas (quita espacios al inicio/fin)
cartera_df.columns = cartera_df.columns.str.strip()

# Conversión de columnas de fecha a datetime
cols_fecha_cartera = [
    "FECHA CANCELACIÓN",
    "FECHA LIQUIDACIÓN INTERÉS",
    "INICIO DIFERIDO",
    "FECHA",
    "FECHA VENCE",
    "FECHA DOCUMENTO",
    "FECHA VENCIMIENTO",
]

for col in cols_fecha_cartera:
    if col in cartera_df.columns:
        cartera_df[col] = pd.to_datetime(
            cartera_df[col],
            errors="coerce",
            dayfirst=True
        )

# Conversión de columnas numéricas ENTERAS (códigos, contadores, etc.)
cols_int_cartera = [
    "ORGANIZACIÓN ORIGEN",
    "NÚMERO ORIGEN",
    "CLASIFICADOR",
    "REFERENCIA",
    "GRUPO",
    "LIQUIDACIÓN ORDEN",
    "CRÉDITO",
    "FUENTE FUNCIÓN",
    "CLIENTE",
    "DÍAS",
    "# REG",
    "EDAD CARTERA",
    "NÚMERO CRÉDITO",
    "ORGANIZACIÓN",
    "TIPO CLIENTE",
    "VALOR DESCUENTO",
    "FONDO",
    "ORGANIZACIÓN CENTRO",
    "CENTRO COSTO",
    "MESES DIFERIDO",
]

for col in cols_int_cartera:
    if col in cartera_df.columns:
        cartera_df[col] = pd.to_numeric(cartera_df[col], errors="coerce").astype("Int64")

# Conversión de columnas numéricas CONTINUAS (montos, tasas, saldos)
cols_float_cartera = [
    "CUOTA",
    "TASA INTERÉS MORA",
    "CONCEPTO NOTA",
    "CAUSA NOTA",
    "SALDO",
    "VALOR BRUTO",
    "VALOR TOTAL",
    "VALOR AFECTADO",
    "VALOR DOCUMENTO",
]

for col in cols_float_cartera:
    if col in cartera_df.columns:
        cartera_df[col] = pd.to_numeric(cartera_df[col], errors="coerce")

# Conversión de columnas categóricas (texto con categorías)

cols_cat_cartera = [
    "DOCUMENTO ORIGEN",
    "NOMBRE REFERENCIA",
    "NOMBRE CONCEPTO CRÉDITO",
    "CONCEPTO CRÉDITO",
    "NOMBRE CAUSA NOTA",
    "CONTROL SALDO",
    "NOMBRE CONCEPTO",
    "NOMBRE PERIODO",
    "NOMBRE FUENTE FUNCIÓN",
    "TIPO ID",
    "ID_Estudiante",
    "NOMBRE EDAD",
    "CONCEPTO",
    "NOMBRE DOCUMENTO",
    "RANGO",
    "NOMBRE TIPO CLIENTE",
    "TIPO CARTERA",
    "TIPO CARTERA 2",
    "ACTIVA / INACTIVA",
    "TIPO DOCUMENTO",
    "NOMBRE TIPO DOCUMENTO",
    "DOCUMENTO",
    "GENERA MORA",
    "DESCRIPCIÓN",
    "NOMBRE CENTRO COSTO",
    "NOMBRE FONDO",
    "ESTADO",
]

for col in cols_cat_cartera:
    if col in cartera_df.columns:
        cartera_df[col] = cartera_df[col].astype("category")

print("Corrección de tipos completada para cartera_df.")


Corrección de tipos completada para cartera_df.


In [44]:
tipos_cartera_post = pd.DataFrame({
    "columna": cartera_df.columns,
    "tipo_dato": cartera_df.dtypes.astype(str)
}).reset_index(drop=True)

estilo_tabla(tipos_cartera_post, titulo="Tipos de datos después de la corrección – Cartera Depurada")


,columna,tipo_dato
0,CLIENTE,Int64
1,TIPO ID,category
2,ID_Estudiante,category
3,# REG,Int64
4,EDAD CARTERA,Int64
5,NOMBRE EDAD,category
6,DÍAS,Int64
7,RANGO,category
8,SALDO,float64
9,ORGANIZACIÓN,Int64


In [45]:
# Corrección de tipos de datos en pagares_df

# Limpieza ligera de nombres de columnas (quita espacios al inicio/fin)
pagares_df.columns = pagares_df.columns.str.strip()

# Conversión de columnas numéricas (Estrato)

cols_int_pagares = [
    "Estrato",
]

for col in cols_int_pagares:
    if col in pagares_df.columns:
        pagares_df[col] = pd.to_numeric(
            pagares_df[col],
            errors="coerce"
        ).astype("Int64")   # entero con soporte para NA

# Conversión de columnas categóricas (incluye IDs y variables socio-demo)

cols_categoricas_pagares = [
    "ID CRÉDITO",      # identificador del crédito
    "ID_CREDITO",      # por si viene sin tilde
    "ID_Estudiante",
    "Sexo",
    "Departamento",
    "Departament",     # por si viene truncado
    "Pais Nacimiento",
    "Pais_Nacimiento",
    "Primiparo",
    "Estado_est",
    "Graduado",
    "Cargo_codeudor",
]

for col in cols_categoricas_pagares:
    if col in pagares_df.columns:
        pagares_df[col] = pagares_df[col].astype("category")

print("Corrección de tipos completada para pagares_df.")
pagares_df.dtypes

info_pagares_tipos = pd.DataFrame({
    "columna": pagares_df.columns,
    "tipo_dato": pagares_df.dtypes.astype(str)
})

# Ordenar alfabéticamente para mejorar legibilidad (opcional)
info_pagares_tipos = info_pagares_tipos.sort_values("columna").reset_index(drop=True)

estilo_tabla(info_pagares_tipos, titulo="Tipos de datos después de la corrección — Info Pagares")



Corrección de tipos completada para pagares_df.


,columna,tipo_dato
0,Cargo_codeudor,category
1,Departamento,category
2,Estado_est,category
3,Estrato,Int64
4,Graduado,category
5,ID CRÉDITO,category
6,ID_Estudiante,category
7,Pais Nacimiento,category
8,Primiparo,category
9,Sexo,category


Primero, cargamos y revisamos los datos para entender qué información trae cada columna. Luego, analizamos el tipo de dato de cada columna (si es número, texto, fecha, etc.) y corregimos aquellos que estaban mal asignados, asegurando que cada variable tenga el formato correcto.

Por ejemplo, convertimos las fechas que estaban como texto a formato de fecha, los montos y cantidades a números, y las columnas con nombres o categorías a un formato especial que permite manejarlas fácilmente. Esto es fundamental para que cualquier análisis, gráfica o modelo que hagamos después sea confiable y funcione correctamente. Así, dejamos los datos listos y organizados para seguir con el análisis exploratorio y la resolución del problema de negocio.

Preparación del EDA:

In [46]:
# Preconfiguración visual para EDA

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams['figure.figsize'] = (10, 5)
sns.set_palette("deep")


Identificación de variables categóricas y numéricas:

In [47]:
# Identificación automática de variables

def identificar_tipos(df):
    numericas = df.select_dtypes(include=["int64", "Int64", "float64"]).columns.tolist()
    fechas    = df.select_dtypes(include=["datetime64[ns]"]).columns.tolist()
    categoricas = df.select_dtypes(include=["category", "object", "string"]).columns.tolist()
    return numericas, fechas, categoricas

# Créditos
num_creditos, fecha_creditos, cat_creditos = identificar_tipos(creditos_df)

# Cartera depurada
num_cartera, fecha_cartera, cat_cartera = identificar_tipos(cartera_df)

# Info Pagares
num_pagares, fecha_pagares, cat_pagares = identificar_tipos(pagares_df)

# Resumen en una sola tabla
resumen_tipos = pd.DataFrame({
    "Variables Numéricas Créditos":   [len(num_creditos)],
    "Variables Categóricas Créditos": [len(cat_creditos)],
    "Variables Fecha Créditos":       [len(fecha_creditos)],
    "Variables Numéricas Cartera":    [len(num_cartera)],
    "Variables Categóricas Cartera":  [len(cat_cartera)],
    "Variables Fecha Cartera":        [len(fecha_cartera)],
    "Variables Numéricas Pagares":    [len(num_pagares)],
    "Variables Categóricas Pagares":  [len(cat_pagares)],
    "Variables Fecha Pagares":        [len(fecha_pagares)],
})

estilo_tabla(resumen_tipos, titulo="Resumen inicial de tipos por dataset")


,Variables Numéricas Créditos,Variables Categóricas Créditos,Variables Fecha Créditos,Variables Numéricas Cartera,Variables Categóricas Cartera,Variables Fecha Cartera,Variables Numéricas Pagares,Variables Categóricas Pagares,Variables Fecha Pagares
0,12,14,4,30,29,7,1,9,0


Estadísticas descriptivas:

In [55]:
# Estadísticas Descriptivas - Créditos

# 1. Excluir Cliente
num_creditos_filtrado = [col for col in num_creditos if col != "Cliente"]

# 2. Estadísticos base
desc_creditos = creditos_df[num_creditos_filtrado].describe().T

# 3. Mediana y rango
desc_creditos["median"] = creditos_df[num_creditos_filtrado].median()
desc_creditos["range"] = desc_creditos["max"] - desc_creditos["min"]

# 4. Reordenar columnas
desc_creditos = desc_creditos[
    ["count", "mean", "median", "std", "min", "25%", "50%", "75%", "max", "range"]
]

# 5. Función de estilo
def estilo_estadisticas(df):
    return (
        df.style
        .set_caption("📊 Estadísticas Descriptivas – Créditos")
        .format("{:,.0f}", na_rep="-")  # SIN decimales + separador de miles
        .set_table_styles([
            # Estilo del título
            {"selector": "caption",
             "props": [("font-size", "18px"),
                       ("font-weight", "bold"),
                       ("color", "#2c3e50"),
                       ("text-align", "left"),
                       ("margin", "10px 0")]},

            # Encabezados
            {"selector": "th",
             "props": [("background-color", "#1f2d3d"),
                       ("color", "white"),
                       ("font-weight", "bold"),
                       ("padding", "8px"),
                       ("border-bottom", "2px solid #34495e")]}
        ])
        .set_properties(**{  # Estilo celdas
            "text-align": "right",
            "padding": "6px",
            "border": "1px solid #e0e0e0",
            "font-size": "13px"
        })
        .apply(lambda x: ["background-color: #f9f9f9" if i % 2 == 0 else ""
                          for i in range(len(x))], axis=0)  # filas alternadas
    )

# Mostrar tabla con estilo
estilo_estadisticas(desc_creditos)





,count,mean,median,std,min,25%,50%,75%,max,range
Credito,"405,829","84,009","71,918","58,036",1,"33,210","71,918","135,086","199,727","199,726"
Contar,"405,829",0,0,0,0,0,0,1,1,1
Nota_debito,"405,829","1,483,930","1,464,173","861,110","10,036","718,773","1,464,173","2,220,639","2,983,837","2,973,801"
Concepto_nota_debito,"405,829",134,99,100,99,99,99,99,515,416
Causa_nota_debito,"405,829",281,204,192,172,204,204,204,870,698
Valor_nota_debito,"405,829","777,171","537,500","988,349",40,"307,798","537,500","903,200","85,572,101","85,572,061"
Valor_pagado_nota_debito,"405,829","726,923","510,000","907,769",0,"279,750","510,000","874,800","85,572,101","85,572,101"
Saldo_nota_debito,"405,829","50,248",0,"442,721",0,0,0,0,"42,099,738","42,099,738"
% Recaudo,"405,829",96,100,20,0,100,100,100,100,100
Centro_costo,"405,829","2,217","2,082",580,"1,016","2,060","2,082","2,152","8,888","7,872"


**Análisis de Estadísticas Descriptivas - Dataset Créditos**

Al revisar las estadísticas de las variables numéricas del dataset de créditos, podemos identificar varios patrones interesantes sobre el comportamiento de los créditos educativos.

**Valor de las notas de débito:**

Los valores de las notas de débito tienen un promedio de 777.171, pero lo interesante es que la mediana es bastante menor (537.500), lo que nos indica que hay algunos créditos con valores muy altos que están elevando el promedio. La mayoría de los créditos se concentra en montos más moderados, con el 50% de ellos entre 307.798 y 903.200. Sin embargo, encontramos casos extremos que llegan hasta los $85.5 millones, lo que sugiere una gran diversidad en los tipos de créditos otorgados.

**Pagos realizados:**

En cuanto a los pagos, vemos un comportamiento similar. El promedio de lo pagado por nota de débito es de 726.923, con una mediana de 510.000. Esto significa que, en general, los estudiantes han estado pagando sus obligaciones, aunque la variabilidad es alta. Lo más notable es que hay notas de débito completamente pagadas (valores que llegan al máximo de 85.5 millones) y otras que aún no han recibido ningún pago (mínimo de 0).

**Saldo pendiente:**

El saldo promedio pendiente es relativamente bajo (50.248), pero aquí hay algo importante: la mediana es 0, lo que significa que más de la mitad de las notas de débito están completamente pagadas. Esto es una señal positiva de recuperación. Sin embargo, hay casos con saldos muy altos (hasta 42 millones), lo que representa un riesgo de cartera que debe monitorearse.

**Porcentaje de recaudo:**

Esta es quizás la métrica más alentadora: el porcentaje promedio de recaudo es del 96%, y la mediana es del 100%. Esto nos dice que la gran mayoría de los créditos educativos están siendo pagados en su totalidad. De hecho, el 75% de los casos tienen un recaudo del 100%, lo que refleja una buena cultura de pago entre los beneficiarios.

**Cantidad de cuotas:**

Los créditos tienen en promedio 8 cuotas, aunque la mediana es de solo 4, lo que indica que muchos créditos son a corto plazo. La mayoría se paga entre 1 y 5 cuotas, aunque hay casos que se extienden hasta 208 cuotas (más de 17 años), probablemente para créditos de mayor monto.

In [58]:
# Variables numéricas reales (Cartera Depurada)
num_cartera_reales = [
    "SALDO",
    "CUOTA",
    "VALOR BRUTO",
    "VALOR DESCUENTO",
    "VALOR TOTAL",
    "VALOR AFECTADO",
    "VALOR DOCUMENTO",
    "TASA INTERÉS MORA",
    "EDAD CARTERA",
    "DÍAS",
    "MESES DIFERIDO"
]

# Filtrar columnas existentes
num_cartera_reales = [col for col in num_cartera_reales if col in cartera_df.columns]

# Estadísticas descriptivas
desc_cartera = cartera_df[num_cartera_reales].describe().T

# Mediana y rango
desc_cartera["median"] = cartera_df[num_cartera_reales].median()
desc_cartera["range"] = desc_cartera["max"] - desc_cartera["min"]

# Ordenar columnas
desc_cartera = desc_cartera[
    ["count", "mean", "median", "std", "min", "25%", "50%", "75%", "max", "range"]
]

# Convertir TODAS las columnas a numéricas (para aplicar formato sin decimales)
for col in desc_cartera.columns:
    desc_cartera[col] = pd.to_numeric(desc_cartera[col], errors="coerce")

# Mostrar tabla con estilo premium y sin decimales
estilo_tabla(desc_cartera, "📊 Estadísticas Descriptivas – Cartera Depurada")



,count,mean,median,std,min,25%,50%,75%,max,range
SALDO,"27,330","829,633","342,985","1,720,974",0,"97,538","342,985","876,600","77,520,000","77,520,000"
CUOTA,"20,835",19,5,24,0,3,5,30,208,208
VALOR BRUTO,"27,330","948,307","446,044","2,534,522",0,"141,480","446,044","922,997","273,709,934","273,709,934"
VALOR DESCUENTO,"27,330",0,0,0,0,0,0,0,0,0
VALOR TOTAL,"27,330","948,307","446,044","2,534,522",0,"141,480","446,044","922,997","273,709,934","273,709,934"
VALOR AFECTADO,"27,330","137,400",0,"1,626,035",0,0,0,0,"224,481,484","224,481,484"
VALOR DOCUMENTO,"27,330","948,307","446,044","2,534,522",0,"141,480","446,044","922,997","273,709,934","273,709,934"
TASA INTERÉS MORA,"25,324",0,0,0,0,0,0,0,2,2
EDAD CARTERA,"27,330",9,9,4,1,7,9,13,14,13
DÍAS,"27,330",450,-5,"1,561","-4,773",-243,-5,584,"9,021","13,794"


**Análisis de Estadísticas Descriptivas - Dataset Cartera Depurada**


El análisis de la cartera depurada nos ofrece una visión más detallada del estado actual de los créditos y su comportamiento de pago.

**Saldo de la cartera:**

El saldo promedio por registro es de 829.633, con una mediana de 342.985, lo que nuevamente indica que hay algunos créditos con saldos muy elevados que jalan el promedio hacia arriba. La mitad de los registros tiene saldos entre 97.538 y 876.600. Los casos extremos llegan hasta 77.5 millones, lo que representa los créditos más grandes y posiblemente más antiguos en la cartera.

**Valor de las cuotas:**

Las cuotas tienen un promedio de 19, pero aquí hay algo curioso: la mediana es de solo 5, lo que sugiere que muchos créditos tienen cuotas pequeñas o están prácticamente liquidados. El rango es amplio, desde 0 hasta 208, reflejando la diversidad en los esquemas de pago.

**Montos totales:**

El valor bruto promedio de los créditos es de 948,307, prácticamente igual al valor total (948.307) y al valor del documento (948.307), lo que tiene sentido porque estos tres campos deberían ser similares. La mediana está alrededor de 446.044, indicando que la mitad de los créditos son de montos moderados, mientras que los más grandes alcanzan los 273 millones.

**Descuentos e intereses de mora:**

Algo muy llamativo es que tanto el valor de descuento como la tasa de interés de mora tienen valores de cero en todas sus estadísticas (media, mediana, percentiles). Esto significa que estas columnas probablemente están vacías o no se están aplicando descuentos ni cobrando intereses moratorios en esta cartera, lo cual es inusual y debería verificarse.

**Valor afectado:**

El valor afectado promedio es de 137.400, pero la mediana es 0, lo que indica que más de la mitad de los créditos no tienen montos afectados (posiblemente por mora o problemas). Sin embargo, hay casos donde el valor afectado llega hasta 224 millones, lo que representa créditos con serios problemas de recuperación.

**Edad de la cartera:**

La cartera tiene en promedio 9 años de antigüedad, con una mediana también de 9 años. Esto indica que estamos trabajando con créditos relativamente antiguos. El 75% de la cartera tiene entre 7 y 13 años, y el crédito más antiguo tiene 14 años. Esta antigüedad puede estar relacionada con la dificultad de recuperación en algunos casos.


**Días de atraso o anticipación:**

Aquí encontramos algo que requiere atención: el promedio de días es 450, pero hay valores negativos (mínimo de -4,773 días), lo que podría indicar pagos anticipados o errores en el cálculo de fechas. La mediana es de -5 días, sugiriendo que muchos pagos se realizan antes de la fecha de vencimiento. Sin embargo, hay casos con atrasos de hasta 9,021 días (casi 25 años), lo cual es extremo y debe investigarse.

**Meses diferidos:**

Los créditos tienen en promedio 1 mes diferido, con una mediana de 0 meses, indicando que la mayoría no tiene períodos de gracia. El rango va de 0 a 10 meses, lo que muestra flexibilidad en algunos casos especiales.

**Conclusión General:**

Ambos datasets muestran una cartera de créditos educativos con un comportamiento mayormente positivo en términos de recaudo, pero con algunos casos extremos que requieren atención especial. La antigüedad de la cartera y la presencia de valores atípicos en días y montos afectados sugieren que hay un segmento de créditos problemáticos que deberían ser el foco de las estrategias de cobranza y gestión de riesgo.

**Análisis de Variables Categóricas (Estadísticas de Frecuencia)**

In [64]:
def resumen_categoricas(df, titulo="Resumen de Variables Categóricas"):
    cat_cols = df.select_dtypes(include=["category", "object", "string"]).columns

    tablas = {}
    for col in cat_cols:
        tablas[col] = (
            df[col]
            .value_counts(dropna=False)
            .to_frame("count")
            .assign(percentage=lambda x: (x["count"] / x["count"].sum() * 100).round(2))
        )

    print(f"🔎 {titulo}: {len(cat_cols)} variables encontradas")
    return tablas

tablas_creditos = resumen_categoricas(creditos_df, "Créditos – Variables Categóricas")
tablas_cartera  = resumen_categoricas(cartera_df, "Cartera Depurada – Variables Categóricas")


🔎 Créditos – Variables Categóricas: 14 variables encontradas
🔎 Cartera Depurada – Variables Categóricas: 29 variables encontradas


Función para generar tablas con estilo:

In [65]:
def tabla_frecuencias(df, col, titulo=None):
    """
    Genera una tabla con:
    - Conteos
    - Porcentajes
    Visualizada con estilo_tabla()
    """
    tabla = (
        df[col]
        .value_counts(dropna=False)
        .to_frame("count")
        .assign(percentage=lambda x: (x["count"] / x["count"].sum() * 100).round(2))
    )

    return estilo_tabla(tabla, titulo if titulo else f"Frecuencias de {col}")


Identificar variables categóricas por dataset:

In [66]:
# Variables categóricas en Créditos
cat_creditos = creditos_df.select_dtypes(include=["category", "object", "string"]).columns.tolist()

# Variables categóricas en Cartera Depurada
cat_cartera = cartera_df.select_dtypes(include=["category", "object", "string"]).columns.tolist()

print("CATEGÓRICAS – Créditos:", len(cat_creditos))
print("CATEGÓRICAS – Cartera Depurada:", len(cat_cartera))


CATEGÓRICAS – Créditos: 14
CATEGÓRICAS – Cartera Depurada: 29
